In [ ]:
# enable huge pages
# echo always >/sys/kernel/mm/transparent_hugepage/enabled

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_defaults()
sns.set_theme(style='whitegrid')

In [3]:
def bench(source='pointers', n=2**20, q=2**20, cc='clang++'):
    !{cc} -std=c++17 -O3 -march=native -D N={n} -D Q={q} {source}.cc -o run
    res = !./run
    update, query, check = map(float, res)
    check = int(check)
    print(f'{source:>18} {cc:<7} {n:<7} {q:<7} {check:10} {update} {query}')
    return update, query

In [44]:
ns = list(int(1.17**k) for k in range(30, 110))

sources = [
    'pointers',
    'topdown-recursive',
    'topdown-iterative',
    # screw topdown-branchless -- it's too hard to implement and explain
    'fenwick',
    'fenwick-holes',
    'bottomup-twoends',
    'bottomup-general',
    'bottomup-branchless',
    'simd'
]

results = {}

In [45]:
for source in sources:
    results[source] = [bench(source, n=n, q=2**22) for n in ns]

          pointers clang++ 111     4194304   30137953 57.87 83.95
          pointers clang++ 129     4194304   12552218 60.84 88.88
          pointers clang++ 152     4194304   22270488 61.2 91.15
          pointers clang++ 177     4194304    3014878 62.87 95.31
          pointers clang++ 208     4194304   19584631 65.85 96.94
          pointers clang++ 243     4194304   18253269 67.89 100.36
          pointers clang++ 284     4194304   28214315 70.19 102.95
          pointers clang++ 333     4194304   26275784 72.93 108.22
          pointers clang++ 389     4194304     982020 76.5 110.63
          pointers clang++ 456     4194304    2349222 80.87 115.57
          pointers clang++ 533     4194304   25956268 83.94 117.74
          pointers clang++ 624     4194304   22596458 89.43 120.97
          pointers clang++ 730     4194304    4305393 94.08 126.91
          pointers clang++ 855     4194304   22974297 97.93 128.96
          pointers clang++ 1000    4194304   22416047 103.32 135.23
 

In [49]:
simd8 = [bench('simd', n=n, q=2**22) for n in ns]

              simd clang++ 111     4194304   30137953 5.47 1.66
              simd clang++ 129     4194304   24809777 5.51 1.66
              simd clang++ 152     4194304   22270488 5.52 1.68
              simd clang++ 177     4194304    3014878 5.47 1.67
              simd clang++ 208     4194304   19584631 5.45 1.66
              simd clang++ 243     4194304   18253269 5.45 1.67
              simd clang++ 284     4194304   28214315 5.52 1.68
              simd clang++ 333     4194304   26275784 5.53 1.66
              simd clang++ 389     4194304   11040013 5.51 1.67
              simd clang++ 456     4194304    2349222 5.48 1.79
              simd clang++ 533     4194304   25623368 5.47 1.67
              simd clang++ 624     4194304   22596458 7.09 2.19
              simd clang++ 730     4194304    4305393 7.08 2.1
              simd clang++ 855     4194304   22974297 7.06 2.08
              simd clang++ 1000    4194304   22416047 7.08 2.1
              simd clang++ 1170    4194304

In [53]:
results.keys()

dict_keys(['pointers', 'topdown-recursive', 'topdown-iterative', 'fenwick', 'fenwick-holes', 'bottomup-twoends', 'bottomup-general', 'bottomup-branchless', 'simd', 'simd8', 'simd32', 'simd64'])

In [52]:
results['simd8'] = simd8
results['simd32'] = simd32
results['simd64'] = simd64

In [54]:
import pickle

with open('results.pkl', 'wb') as file:
    pickle.dump(results, file)

In [8]:
def plot(title=None, ylabel=None, path=None, ch=None, legend=True, ylim=None):
    if ch:
        lines = [
            (2**13, "32K"),
            (2**17, "512K"),
            (2**20, "4M"),
        ]

        for x, t in lines:
            plt.text(x * 1.2, ch, t)
            plt.axvline(x=x, color='black', linestyle='--')

    plt.xscale('log', basex=2)
    plt.xlabel('Array size')

    if ylabel:
        plt.ylabel(ylabel)

    if title:
        plt.title(title, pad=12)

    if legend:
        plt.legend(loc='best')

    plt.ylim(bottom=0, top=ylim)
    plt.margins(0)

    if path:
        fig = plt.gcf()
        fig.savefig(path)
    
    plt.show()